In [1]:
import pandas as pd
import os 
current_dir = os.getcwd()
from sklearn.preprocessing import StandardScaler

In [2]:
pd.set_option('display.max_columns', None)
current_dir = os.getcwd()

#### Leer solo la columna de fechas primero

In [3]:
total_data_path = os.path.join(current_dir, '../data/raw/total_data.csv')

fechas = pd.read_csv(total_data_path, usecols=["date"])

#### Ver cuántas fechas únicas hay

In [4]:
fechas_unicas = fechas["date"].nunique()
print(f"Fechas únicas: {fechas_unicas}")

Fechas únicas: 1941


####  Leer por chunks y seleccionar 500 filas por fecha

In [5]:

output = []

# Primero descubrimos qué columnas tiene (para leer todo después por partes)
columnas = pd.read_csv(total_data_path, nrows = 1).columns

# Creamos un diccionario para contar cuántas filas hemos tomado por fecha
cuenta_fechas = {}

# Queremos 1 millón de filas
meta_total = 1_000_000
chunk_size = 100_000
total_actual = 0

for chunk in pd.read_csv(total_data_path, chunksize = chunk_size):
    for fecha, grupo in chunk.groupby("date"):
        if cuenta_fechas.get(fecha, 0) < 500:
            faltan = 500 - cuenta_fechas.get(fecha, 0)
            seleccion = grupo.sample(n = min(faltan, len(grupo)), random_state = 77)
            output.append(seleccion)
            cuenta_fechas[fecha] = cuenta_fechas.get(fecha, 0) + len(seleccion)
            total_actual += len(seleccion)
            
            if total_actual >= meta_total:
                break
    if total_actual >= meta_total:
        break

C:\Users\andta\AppData\Local\Temp\ipykernel_7732\1973051828.py:14: DtypeWarning: Columns (14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(total_data_path, chunksize = chunk_size):
C:\Users\andta\AppData\Local\Temp\ipykernel_7732\1973051828.py:14: DtypeWarning: Columns (14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(total_data_path, chunksize = chunk_size):
C:\Users\andta\AppData\Local\Temp\ipykernel_7732\1973051828.py:14: DtypeWarning: Columns (14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(total_data_path, chunksize = chunk_size):
C:\Users\andta\AppData\Local\Temp\ipykernel_7732\1973051828.py:14: DtypeWarning: Columns (14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(total_data_path, chunksize = chunk_size):
C:\Users\andta\AppData\Local\Temp\ipyker

#### Concatenar y guardar en csv (1 Millon de filas)

In [6]:
final_data = pd.concat(output)

final_data_path = os.path.join(current_dir, '../data/raw/final_data.csv')

final_data.to_csv(final_data_path, index=False)

In [7]:
final_data.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d,sales,date,wm_yr_wk,weekday,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price
3573,HOBBIES_2_109_CA_2_evaluation,HOBBIES_2_109,HOBBIES_2,HOBBIES,CA_2,CA,d_1,0,2011-01-29,11101,Saturday,1,1,2011,NaN,NaN,NaN,NaN,0,0,0,NaN
3529,HOBBIES_2_065_CA_2_evaluation,HOBBIES_2_065,HOBBIES_2,HOBBIES,CA_2,CA,d_1,0,2011-01-29,11101,Saturday,1,1,2011,NaN,NaN,NaN,NaN,0,0,0,NaN
24983,HOUSEHOLD_1_027_WI_2_evaluation,HOUSEHOLD_1_027,HOUSEHOLD_1,HOUSEHOLD,WI_2,WI,d_1,0,2011-01-29,11101,Saturday,1,1,2011,NaN,NaN,NaN,NaN,0,0,0,5.36
20557,FOODS_3_038_TX_3_evaluation,FOODS_3_038,FOODS_3,FOODS,TX_3,TX,d_1,0,2011-01-29,11101,Saturday,1,1,2011,NaN,NaN,NaN,NaN,0,0,0,NaN
4070,HOUSEHOLD_1_466_CA_2_evaluation,HOUSEHOLD_1_466,HOUSEHOLD_1,HOUSEHOLD,CA_2,CA,d_1,0,2011-01-29,11101,Saturday,1,1,2011,NaN,NaN,NaN,NaN,0,0,0,NaN


#### Revision columnas

In [8]:
print(final_data.id.nunique())
print(final_data.id.isnull().sum())
print(final_data.id.dtype)

26670
0
object


In [9]:
with open("resumen_columnas.txt", "w") as f:
    for col in final_data.columns:
        unicos = final_data[col].nunique()
        nulos = final_data[col].isnull().sum()
        tipo = final_data[col].dtype

        f.write(f"Columna: {col}\n")
        f.write(f"  Valores únicos: {unicos}\n")
        f.write(f"  Valores nulos: {nulos}\n")
        f.write(f"  Tipo de dato: {tipo}\n")
        f.write("\n")


### Procesamiento de los datos

#### Preprocesamiento de la columna `date`

In [10]:
# Convertir la columna 'date' a formato datetime
final_data['date'] = pd.to_datetime(final_data['date'])

# Extraer características de la fecha
final_data['day_of_week'] = final_data['date'].dt.dayofweek  # Día de la semana (0 = lunes, 6 = domingo)
final_data['is_weekend'] = final_data['day_of_week'].isin([5, 6]).astype(int)  # Fin de semana (1 = sí, 0 = no)
final_data['month'] = final_data['date'].dt.month  # Mes
final_data['quarter'] = final_data['date'].dt.quarter  # Trimestre


### Procesamiento de valores nulos

In [11]:
# Rellenar valores nulos con 0 o con el valor medio de la columna
final_data['event_name_1'].fillna('No Event', inplace=True)
final_data['event_type_1'].fillna('No Type', inplace=True)
final_data['event_name_2'].fillna('No Event', inplace=True) 
final_data['event_type_2'].fillna('No Type', inplace=True)

C:\Users\andta\AppData\Local\Temp\ipykernel_7732\1541185424.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  final_data['event_name_1'].fillna('No Event', inplace=True)
C:\Users\andta\AppData\Local\Temp\ipykernel_7732\1541185424.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

In [12]:
# Rellenar valores nulos en 'sell_price' con el último valor conocido para cada item_id y store_id
final_data['sell_price'] = final_data.groupby(['item_id', 'store_id'])['sell_price'].ffill()

# Verificamos que no haya más valores nulos
print(final_data['sell_price'].isnull().sum())



198189


In [13]:
# Rellenar valores nulos en 'sell_price' con el siguiente valor conocido para cada item_id y store_id
final_data['sell_price'] = final_data.groupby(['item_id', 'store_id'])['sell_price'].bfill()

# Verificamos que no haya más valores nulos
print(final_data['sell_price'].isnull().sum())


2855


In [14]:
# Rellenar los valores nulos restantes con la media de sell_price
mean_sell_price = final_data['sell_price'].mean()
final_data['sell_price'].fillna(mean_sell_price, inplace=True)

# Verificamos que no haya más valores nulos
print(final_data['sell_price'].isnull().sum())


0


C:\Users\andta\AppData\Local\Temp\ipykernel_7732\1176023560.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  final_data['sell_price'].fillna(mean_sell_price, inplace=True)


### Escalar la variable `sell_price`

In [15]:
scaler = StandardScaler()
final_data['scaled_sell_price'] = scaler.fit_transform(final_data[['sell_price']])

In [16]:
final_data.date.min()

Timestamp('2011-01-29 00:00:00')

In [17]:
final_data.date.max()

Timestamp('2016-05-22 00:00:00')

In [18]:
final_data.state_id.value_counts()

state_id
CA    496057
TX    256552
WI    217891
Name: count, dtype: int64